### Observation data resources in Uinta Basin

In [1]:
import datetime
import io
from urllib.request import urlopen, Request
import sqlite3

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import pandas as pd 

from synoptic.plots import map_metadata
import synoptic.services as ss
%matplotlib inline

# Re-do SQL if not already done - overwriting for testing
already_done = False


/Users/johnlawson/anaconda3/envs/live-verification/lib/python3.11/site-packages/synoptic/accessors.py:28: UserWarning: map making not available without Brian's cartopy_tools
  warnings.warn("map making not available without Brian's cartopy_tools")


In [2]:
def image_spoof(self, tile):
    """Thanks to Joshua Hrisko 
    https://makersportal.com/blog/2020/4/24/geographic-visualizations-in-python-with-cartopy
    """    
    # get the url of the street map API
    url = self._image_url(tile) 
    req = Request(url) 
    req.add_header('User-agent','Anaconda 3') 
    fh = urlopen(req)
    im_data = io.BytesIO(fh.read()) 
    fh.close() 
    # open image with PIL, set format, reformat for Cartopy
    img = Image.open(im_data) 
    img = img.convert(self.desired_tile_form)
    return (img, self.tileextent(tile), 'lower') 

In [3]:
# Terrain and roads etc
radius = "KVEL,50"
recent = 180
cimgt.Stamen.get_image = image_spoof  # reformat web request for street map spoofing
osm_img = cimgt.Stamen('terrain')  # spoofed, downloaded street map

df_meta = ss.stations_metadata(radius=radius, recent=recent)
df_meta

Checking for data recent=120 minutes.

 🚚💨 Speedy Delivery from Synoptic API [metadata]: https://api.synopticdata.com/v2/stations/metadata?radius=KVEL,50&recent=120&token=🙈HIDDEN


,KVEL,RSVL,RNGC2,CWHU1,HIRU1,HPSU1,LKFU1,MMTU1,TCKU1,SURC2,...,UTWIS,UTBAS,UTMOP,UT43W,JESU1,PC101,4276P,PC266,PC353,K40U
DISTANCE,0.0,31.68,46.06,34.74,48.37,39.79,49.51,22.78,22.12,30.77,...,28.52,19.56,28.45,41.71,14.81,7.43,10.98,11.98,4.68,38.39
ELEVATION,5262.0,5151.0,5290.0,10498.0,9167.0,9121.0,10127.0,9552.0,9448.0,5960.0,...,7927.0,8368.0,8076.0,7231.0,4776.0,5855.0,4803.0,8426.0,5947.0,6175.0
ELEV_DEM,5269.0,5200.1,5295.3,10524.9,9179.8,9137.1,10157.5,9583.3,9471.8,6282.8,...,7923.2,8366.1,8084.0,7224.4,4780.2,NaN,NaN,8441.6,NaN,NaN
ID,61,878,964,1458,1472,1473,1482,1494,1509,1969,...,151978,151979,151983,151984,170574,172295,173578,175633,217351,218094
MNET_ID,1,1005,106,25,25,25,25,25,25,2,...,4,4,4,4,23,247,208,247,247,1
NAME,Vernal Airport,ROOSEVELT-UNION HS,RANGLEY 1E (POWER PLANT),CHEPETA,HOLE-IN-ROCK,HICKERSON PARK,LAKEFORK #1,MOSBY MTN.,TROUT CREEK,DINOSAUR NM SUCCESS,...,US-191 at MP 386 Willie Spring,US-191 at MP 377 Bassett Spring,SR-44 at MP 9 Moose Pond,SR-43 at MP 0.5 West Stateline,GREEN RIVER NEAR JENSEN 6ENE,Dry Fork Canyon,Dino Ozone Station,Little Mountain,Vernal West,MANILA
RECORD_END,2023-11-17 17:10:00+00:00,2002-02-13 21:00:00+00:00,2023-10-21 03:00:00+00:00,2023-11-17 16:00:00+00:00,2023-11-17 16:00:00+00:00,2023-11-17 16:00:00+00:00,2023-11-17 15:00:00+00:00,2023-11-17 16:00:00+00:00,2023-11-17 16:00:00+00:00,2023-11-17 16:22:00+00:00,...,2023-11-17 17:10:00+00:00,2023-11-17 17:10:00+00:00,2023-11-17 17:10:00+00:00,2023-11-17 17:10:00+00:00,2023-11-17 15:30:00+00:00,2023-11-17 17:10:00+00:00,2022-05-27 03:10:00+00:00,2023-11-17 17:10:00+00:00,2023-11-17 17:10:00+00:00,2023-11-17 16:55:00+00:00
RECORD_START,1997-01-01 00:00:00+00:00,1999-01-12 00:00:00+00:00,1997-01-01 00:00:00+00:00,2000-01-20 00:00:00+00:00,2000-01-20 00:00:00+00:00,2000-01-20 00:00:00+00:00,2000-01-20 00:00:00+00:00,2000-01-20 00:00:00+00:00,2000-01-20 00:00:00+00:00,1998-07-30 00:00:00+00:00,...,2020-06-04 17:10:00+00:00,2020-06-04 17:10:00+00:00,2020-06-05 00:40:00+00:00,2020-06-05 00:40:00+00:00,2021-10-29 15:41:00+00:00,2022-03-11 08:14:00+00:00,2022-04-26 21:10:00+00:00,2022-06-30 20:15:00+00:00,2023-05-16 08:14:00+00:00,2023-05-24 20:23:00+00:00
RESTRICTED,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
STATE,UT,UT,CO,UT,UT,UT,UT,UT,UT,CO,...,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT


I would like to see maps that show available stations for each variable (e.g., temperature, wind, etc). The aim is to have a dataset to run mining and neural networks on to find:

* Areas of most sensitivity for, e.g., central Basin towns' weather conditions
* Clusters in the dataset that might automatically "detect" inversion events
* Are there areas with low observational coverage? South end of Basin; exit canyons?
* Will need to build dataset for a number of NDJF periods (?) and fill gaps of missing data 
* I want to generate criteria in weather observations for inversion onset, persistence, and dissipation

In [8]:
# Show stations with temperature, wind direction, wind speed, humidity 
def get_density(P,T,Td):
    """
    P in Pa
    T in K
    Td in K 
    """
    R_d = 287.05  # J/(kg·K)
    R_v = 461.495  # J/(kg·K)
    
    # Celsius for this
    Td_C = Td - 273.15
    
    # Calculate partial pressure of water vapour
    P_v = 6.11 * 10 ** (7.5 * Td_C / (237.3 + Td_C))
    
    # Calculate partial pressure of dry air
    P_d = P - P_v
    
    # Calculate air density
    rho = (P_d / (R_d * T)) + (P_v / (R_v * T))
    return rho

vars=["wind_speed","wind_direction","air_temp","dew_point_temperature",
            "pressure","snow_depth","solar_radiation",
            "relative_humidity","wind_gust","altimeter","soil_temp",
            "sea_level_pressure","snow_accum", "road_temp",
            "cloud_layer_1_code", "cloud_layer_2_code",
            "cloud_layer_3_code", "cloud_low_symbol",
            "cloud_mid_symbol", "cloud_high_symbol",
            "sonic_wind_direction", "peak_wind_speed",
            "ceiling", "sonic_wind_speed", "soil_temp_ir",
            "snow_smoothed", "snow_accum_manual", "snow_water_equiv",
            "precipitable_water_vapor", "net_radiation_sw",
            "sonic_air_temp", "sonic_vertical_vel",
            "vertical_heat_flux", "outgoing_radiation_sw",
            "PM_25_concentration", "ozone_concentration",
            "derived_aerosol_boundary_layer_depth",
            "NOx_concentration", "PM_10_concentration",
            "visibility_code", "cloud_layer_1", "cloud_layer_2",
            "cloud_layer_3", "wet_bulb_temperature"
            ]
# Let's try KVEL time series and computed density of air 
# stid takes a tuple of station IDs and returns a list of dataframes
df = ss.stations_timeseries(stid="KVEL",start=datetime.datetime(2023,9,1,0,0,0),end=datetime.datetime(2023,9,25,0,0,0),
                                vars=vars)
rho = get_density(df["pressure"],df["air_temp"]+273.15,df["dew_point_temperature"]+273.15)
df = df.assign(air_density=rho.values)
df



 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=KVEL&start=202309010000&end=202309250000&vars=wind_speed,wind_direction,air_temp,dew_point_temperature,pressure,snow_depth,solar_radiation,ozone_concentration,relative_humidity,wind_gust,altimeter,soil_temp,sea_level_pressure,snow_accum,road_temp,cloud_layer_1_code,cloud_layer_2_code,cloud_layer_3_code,cloud_low_symbol,cloud_mid_symbol,cloud_high_symbol,sonic_wind_direction,peak_wind_speed,ceiling,sonic_wind_speed,soil_temp_ir,snow_smoothed,snow_accum_manual,snow_water_equiv,precipitable_water_vapor,net_radiation_sw,sonic_air_temp,sonic_vertical_vel,vertical_heat_flux&token=🙈HIDDEN


,air_temp,altimeter,ceiling,cloud_layer_1_code,cloud_layer_2_code,cloud_layer_3_code,dew_point_temperature_set_1,dew_point_temperature,peak_wind_speed,pressure,relative_humidity,sea_level_pressure_set_1,sea_level_pressure,wind_direction,wind_gust,wind_speed,wind_u,wind_v,air_density
date_time,,,,,,,,,,,,,,,,,,,
2023-09-01 00:00:00+00:00,33.0,101354.563,NaN,1.0,NaN,NaN,NaN,-1.76,NaN,83499.00,10.52,NaN,99564.56,280.0,NaN,3.601,3.546,-0.625,0.950121
2023-09-01 00:05:00+00:00,33.0,101354.563,NaN,1.0,NaN,NaN,NaN,-1.76,NaN,83499.00,10.52,NaN,99564.56,260.0,NaN,2.572,2.533,0.447,0.950121
2023-09-01 00:10:00+00:00,33.0,101354.563,NaN,1.0,NaN,NaN,NaN,0.21,NaN,83499.00,12.18,NaN,99564.47,260.0,NaN,2.572,2.533,0.447,0.950117
2023-09-01 00:15:00+00:00,33.0,101354.563,NaN,1.0,NaN,NaN,NaN,-0.78,NaN,83499.00,11.32,NaN,99564.52,270.0,NaN,3.087,3.087,0.000,0.950119
2023-09-01 00:20:00+00:00,33.0,101354.563,NaN,1.0,NaN,NaN,NaN,-0.78,NaN,83499.00,11.32,NaN,99564.52,270.0,NaN,2.572,2.572,0.000,0.950119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-24 23:45:00+00:00,23.0,102235.024,NaN,1.0,NaN,NaN,NaN,-3.99,NaN,84224.35,16.22,NaN,101017.69,80.0,NaN,1.543,-1.520,-0.268,0.990740
2023-09-24 23:50:00+00:00,23.0,102235.024,NaN,1.0,NaN,NaN,NaN,-3.99,NaN,84224.35,16.22,NaN,101017.69,60.0,NaN,2.058,-1.782,-1.029,0.990740
2023-09-24 23:53:00+00:00,22.8,102235.024,NaN,1.0,NaN,NaN,-4.4,-4.39,NaN,84224.35,15.93,101860.0,101029.90,70.0,NaN,1.543,-1.450,-0.528,0.991410


In [5]:
df.attrs

{'ID': 61,
 'STID': 'KVEL',
 'NAME': 'Vernal Airport',
 'ELEVATION': 5262,
 'STATUS': 'ACTIVE',
 'MNET_ID': 1,
 'STATE': 'UT',
 'TIMEZONE': 'America/Denver',
 'ELEV_DEM': 5269,
 'PERIOD_OF_RECORD': {'start': '1997-01-01T00:00:00Z',
  'end': '2023-11-17T17:35:00Z'},
 'UNITS': {'position': 'm',
  'elevation': 'ft',
  'air_temp': 'Celsius',
  'wind_direction': 'Degrees',
  'wind_speed': 'm/s',
  'dew_point_temperature': 'Celsius',
  'pressure': 'Pascals',
  'wind_u': 'm/s',
  'wind_v': 'm/s'},
 'QC_FLAGGED': False,
 'RESTRICTED': False,
 'RENAMED': {'air_temp_set_1': 'air_temp',
  'dew_point_temperature_set_1': 'dew_point_temperature_set_1',
  'dew_point_temperature_set_1d': 'dew_point_temperature',
  'pressure_set_1d': 'pressure',
  'wind_direction_set_1': 'wind_direction',
  'wind_speed_set_1': 'wind_speed',
  'wind_u_set_1': 'wind_u',
  'wind_v_set_1': 'wind_v'},
 'latitude': 40.44295,
 'longitude': -109.51273,
 'SUMMARY': {'NUMBER_OF_OBJECTS': 1,
  'RESPONSE_CODE': 1,
  'RESPONSE_MESS

In [6]:
# Drop everything that isn't this? Need to add these to a SQL database
conn = sqlite3.connect("observation_repo.db")
cursor = conn.cursor()

# Here we need to set out all variables we like 
# Delete table in testing 
# To avoid loads of API calls, we need a function to create a new column if needed
# cursor.execute("DROP TABLE IF EXISTS weather_data")

# We don't want pressure or altimeter but mslp
cursor.execute("""
    CREATE TABLE IF NOT EXISTS weather_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date_time TEXT,
        stid TEXT,
        elevation REAL,
        latitude REAL,
        longitude REAL,
        air_temp REAL,
        dew_point_temperature REAL,
        wind_direction REAL,
        wind_speed REAL,
        wind_u REAL,
        wind_v REAL,
        snow_depth REAL,
        solar_radiation REAL,
        ozone_concentration REAL,
        air_density REAL
    );"""
)
conn.commit()

In [7]:
# stids.values
# Create lookup for station v elevation, latitude, longitude
# store dict with station v metadata dict 

STID_META = dict()
for stid in stids:
    df_meta = ss.stations_metadata(stid=stid)
    STID_META[stid] = df_meta

NameError: name 'stids' is not defined

In [ ]:
# For each station in stids, let's fetch the dataframe. We will have a list of them ready to concatenate or feed into the SQL database
if already_done is False:
    for stid in stids.values:
        try:
            df = ss.stations_timeseries(stid=stid,start=datetime.datetime(2022,9,26,0,0,0),end=datetime.datetime(2023,9,25,0,0,0),
                                    vars=vars,
                                        verbose=False)
        except AssertionError:
            print("Skipping",stid)
            continue

        stid_lat = STID_META[stid]["latitude"]
        stid_lon = STID_META[stid]["longitude"]
            
        if ("pressure" in df.columns) and ("air_temp" in df.columns) and ("dew_point_temperature" in df.columns):
            rho = get_density(df["pressure"],df["air_temp"]+273.15,df["dew_point_temperature"]+273.15)
            df = df.assign(air_density=rho.values)
        df = df.assign(stid=stid, elevation=df_meta[stid].ELEVATION*0.304,latitude=latitudes,longitude=longitudes)
        
        df.loc[df[stid] == stid, 'latitude'] = stid_lat
        df.loc[df[stid] == stid, 'longitude'] = stid_lon
        
        # Drop everything that isn't in...
        good_cols = vars + ["date_time","wind_u","wind_v","air_density","stid","elevation","latitude","longitude"]
        df.drop(columns=[col for col in df if col not in good_cols], inplace=True)
    
        # pd.to_datetime(df['date_time']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
        pd.to_datetime(df.index.strftime('%Y-%m-%dT%H:%M:%SZ'))
        df.to_sql('weather_data', conn, if_exists='append', index=True, index_label="date_time")
    
    conn.commit()
        # break

In [ ]:
df

In [ ]:
# conn = sqlite3.connect("observation_repo.db")

query = "SELECT * FROM weather_data" # LIMIT 200"

df_q = pd.read_sql_query(query, conn)
display(df_q)  # This will display the DataFrame in Jupyter's rich display system

In [ ]:
def reduce_pressure_to_mslp(P, T, h, L=0.0065):
    """
    Reduce pressure to mean sea level using the hypsometric equation.
    
    Parameters:
    P : float
        Observed pressure at altitude in hPa
    T : float
        Observed temperature at altitude in Celsius
    h : float
        Altitude in meters
    L : float, optional
        Temperature lapse rate in K/m, default is 0.0065 K/m
    
    Returns:
    float
        Pressure at sea level in hPa
    """
    if (np.nan in P) or (np.nan in T) or (np.nan in h):
        return np.nan
    ### TODO: lapse rate, humidity in air...
    R = 287  # Gas constant for dry air in J/(kg·K)
    g = 9.81  # Acceleration due to gravity in m/s^2
    T_kelvin = T + 273.15  # Convert temperature to Kelvin
    
    P0 = P * (1 + (L * h) / T_kelvin)**(g / (L * R))
    
    return P0


In [ ]:
fig = plt.figure(figsize=(12, 9))  # open matplotlib figure
ax = plt.axes(projection=osm_img.crs)  # project using coordinate reference system (CRS) of street map

lats = df_meta.loc["latitude"]
lons = df_meta.loc["longitude"]
stids = df_meta.loc["STID"]
elevs = df_meta.loc["ELEVATION"]*0.304

sc = ax.scatter(lons, lats, c=elevs, transform=ccrs.PlateCarree())#, **scatter_kwargs)
if True:
    for lon, lat, stid, elev in zip(lons, lats, stids, elevs):
        # Colour-code by altitude?
        # sc = ax.scatter(lon,lat,stid,transform=ccrs.PlateCarree(),c=elev)
        # ax.text(lon, lat, stn, transform=ccrs.PlateCarree(),)# **text_kwargs)
        pass
    
cbar = fig.colorbar(sc)

# Add reference towns in RED
towns = {
    "Vernal":[40.4555,-109.5287],
    "Roosevelt":[40.2994,-109.9888],
}
for town,latlon in towns.items():
    ax.scatter(latlon[1], latlon[0], color='red',transform=ccrs.PlateCarree())#, **scatter_kwargs)
    ax.text(latlon[1],latlon[0],town,color='red',transform=ccrs.PlateCarree())

ax.add_feature(cfeature.STATES.with_scale("10m"))
ax.add_feature(cfeature.RIVERS.with_scale("10m"))
# ax.stock_img()
# ax.add_feature(cfeature.NaturalEarthFeature("physical","",scale="10m"))

extent = [-110.6,-108.7,39.65,41.05]
ax.set_extent(extent)  # set extents

# empirical solve for scale based on zoom
scale = np.ceil(-np.sqrt(2) * np.log(np.divide((extent[1] - extent[0]) / 2.0, 350.0)))
scale = (scale < 20) and scale or 19  # scale cannot be larger than 19
ax.add_image(osm_img, int(scale))  # add OSM with zoom specification

ax.set_title("Station Locations", loc="left", fontweight="bold")
fig.tight_layout()
# fig.show()

In [ ]:
# DATA EXPLORATION
# First, a look at the distribution of various things
import seaborn as sns
from matplotlib import rcParams

# Set Helvetica as the font
rcParams['font.family'] = 'Helvetica'

# Sample dataframe (replace this with your actual dataframe)
# df_q = pd.read_csv("your_data.csv")
df_q['date_time'] = pd.to_datetime(df_q['date_time'])

df_x = df_q.copy(deep=True)

# Calculate wind speed
wind_speed = np.sqrt(df_q['wind_u']**2 + df_q['wind_v']**2)
wind_direction = np.degrees(np.arctan2(df_q['wind_v'], df_q['wind_u']))

# Adjust wind direction to meteorological coordinates (0-360 degrees, measured clockwise from true north)
wind_direction = (wind_direction + 360) % 360
 
# We need to re-do the obs table - first let's just convert elevation
mslp = reduce_pressure_to_mslp(df_x["pressure"]/100, df_x["air_temp"], df_x["elevation"])


# deep copy dataframe 
# df_x
# assign these new columns 
df_x = df_x.assign(wind_speed=wind_speed, wind_direction=wind_direction, mslp=mslp)


In [ ]:
import math

def plot_distributions(df, columns_to_plot, title):
    num_plots = len(columns_to_plot)
    num_rows = math.ceil(num_plots / 2)  # Calculate the number of rows needed
    
    fig, axes = plt.subplots(num_rows, 2, figsize=(10, 5 * num_rows))
    fig.suptitle(title)
    
    # Flatten the 2D array of axes if there's more than one row
    axes = axes.flatten() if num_rows > 1 else axes
    # vars=["wind_speed","wind_direction","air_temp","dew_point_temperature","pressure","snow_depth","solar_radiation","ozone_concentration"]

    x_lims = {
                "wind_direction":[1,360],
                "wind_speed":[0,30],
                "air_temp":[-40,40],
                "dew_point_temperature":[-40,20],
                "mslp":[950,1050],
                "pressure":[75000,87500],
                "elevation":[1000,5000],
                # "snow_depth":[0,1],
                "solar_radiation":[0,200],
                "ozone_concentration":[0,150]
    }
    
    for i, col in enumerate(columns_to_plot):
            
        sns.histplot(df[col], kde=True, ax=axes[i])
        axes[i].set_title(col)
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')
        
        if col in x_lims.keys():
            axes[i].set_xlim(x_lims[col])
        
    # Remove any unused subplots
    for i in range(num_plots, num_rows * 2):
        fig.delaxes(axes[i])
        
    plt.tight_layout(rect=(0, 0.03, 1, 0.95))
    plt.show()


In [ ]:
columns_to_plot = ["wind_direction", "wind_speed", "air_temp", "dew_point_temperature", "mslp", "pressure", "elevation", "snow_depth",
                            "solar_radiation","ozone_concentration"]

# Plot for the entire dataframe
plot_distributions(df_x, columns_to_plot+["elevation",], 'Distribution of Variables for Entire Dataset')

In [ ]:
# Extract the month from the date_time column
df_x['month'] = df_x['date_time'].dt.month

# Split the dataframe based on months
df_winter_spring = df_x[df_x['month'].isin([12, 1, 2, 3])]
df_other_months = df_x[~df_x['month'].isin([12, 1, 2, 3])]

# Plot for the winter and spring months
plot_distributions(df_winter_spring, columns_to_plot, 'Distribution of Variables for Dec-Jan-Feb-Mar')


In [ ]:
unique_stid = df_q["stid"].unique()
print(unique_stid)

def plot_stid_locs(stids):
    
    fig = plt.figure(figsize=(12, 9))  # open matplotlib figure
    ax = plt.axes(projection=osm_img.crs)  # project using coordinate reference system (CRS) of street map
    
    _lats = []
    _lons = []
    for s in stids:
        lon = lons[s]
        lat = lats[s]
        elev = elevs[s]
    
        ax.text(lon, lat, s, transform=ccrs.PlateCarree(),)# **text_kwargs)
        _lats.append(lat)
        _lons.append(lon)
    sc = ax.scatter(lons,lats,transform=ccrs.PlateCarree(),c=elevs)
    cbar = fig.colorbar(sc)
    
    ax.add_feature(cfeature.STATES.with_scale("10m"))
    ax.add_feature(cfeature.RIVERS.with_scale("10m"))
    # ax.stock_img()
    # ax.add_feature(cfeature.NaturalEarthFeature("physical","",scale="10m"))
    
    extent = [-110.6,-108.7,39.65,41.05]
    ax.set_extent(extent)  # set extents
    
    # empirical solve for scale based on zoom
    scale = np.ceil(-np.sqrt(2) * np.log(np.divide((extent[1] - extent[0]) / 2.0, 350.0)))
    scale = (scale < 20) and scale or 19  # scale cannot be larger than 19
    ax.add_image(osm_img, int(scale))  # add OSM with zoom specification
    
    ax.set_title("Station Locations", loc="left", fontweight="bold")
    fig.tight_layout()
    # fig.show()
    
plot_stid_locs(unique_stid)

In [ ]:

# Plot for the other months
plot_distributions(df_other_months, columns_to_plot, 'Distribution of Variables for Remaining Months')

In [ ]:
### Could use only obs that are low in the basin by subsetting dataframe

### Subset by ozone events exceeding x and perform PCA

### Can we find "inversion-ness" in the leading PCs?
df_winter_spring

In [ ]:
from sklearn.preprocessing import StandardScaler

# TODO: once I add ozone time series, it will be amazing to compute PCs again
# vars=["wind_speed","wind_direction","air_temp","dew_point_temperature","pressure","snow_depth","solar_radiation","ozone_concentration"]

# pca_cols = ["date_time","wind_speed", "air_temp", "dew_point_temperature", "mslp", "elevation", "snow_depth","solar_radiation","ozone_concentration"]
# pca_cols = ["date_time","wind_speed","air_temp","dew_point_temperature","mslp","elevation"] + ["snow_depth",]
# pca_cols = ["date_time","wind_speed","air_temp","dew_point_temperature","mslp","elevation"] + ["solar_radiation",]
# pca_cols = ["date_time","wind_speed","air_temp","dew_point_temperature","mslp"] + ["ozone_concentration",]
pca_cols = ["date_time","wind_speed","air_temp","mslp","solar_radiation","ozone_concentration"]

# df = df_winter_spring[pca_cols].dropna()[df_winter_spring["elevation"]<2000]
df = df_winter_spring[pca_cols].dropna()[df_winter_spring["elevation"]<1600]

start_time = '2022-12-01 00:00:00'
end_time = '2023-03-31 00:00:00'
df = df[(df['date_time'] >= start_time) & (df['date_time'] <= end_time)]

display(df)

pca_cols.remove('date_time')
# pca_cols = ["wind_speed", "air_temp", "dew_point_temperature", "mslp",]
df = df[pca_cols]

scaler = StandardScaler()
# scaled_data = scaler.fit_transform(df.drop(columns="date_time"))
scaled_data = scaler.fit_transform(df)
# Now perform PCA
from sklearn.decomposition import PCA

# Create a PCA object
pca = PCA(n_components=5)  # Here, we are considering the first three principal components

# Fit the scaled data
principal_components = pca.fit_transform(scaled_data)

# Visualise 
explained_variance = pca.explained_variance_ratio_
plt.bar(range(len(explained_variance)), explained_variance, alpha=0.5, align='center')
plt.title('Variance Contributions from Each Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')
plt.show()

# Leading three components
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(principal_components[:, 0], principal_components[:, 1], principal_components[:, 2])
ax.set_xlabel('First Principal Component')
ax.set_ylabel('Second Principal Component')
ax.set_zlabel('Third Principal Component')
plt.title('Leading Three Principal Components')
plt.show()


In [ ]:
# Get the PCA components (loadings)
loadings = pca.components_

# Create DataFrame from the loadings
loadings_df = pd.DataFrame(loadings.T, columns=[f'PC{i+1}' for i in range(loadings.shape[0])], index=df.columns)

# Create bar chart
ax = loadings_df.plot(kind='bar', figsize=(20, 10))
ax.set_title('Variable Contributions to Principal Components')
ax.set_ylabel('Loading Score')
ax.set_xlabel('Variables')
plt.show()

# Convert principal components to DataFrame
pc_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(principal_components.shape[1])])

# Plot the distribution of each principal component
for col in pc_df.columns:
    sns.histplot(pc_df[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel('Principal Component Score')
    plt.ylabel('Frequency')
    plt.show()
